In [1]:
# Spectacle
import spectacle
from spectacle.core.spectra import Spectrum1D
from spectacle.modeling.models import Absorption1D
from spectacle.core.lines import Line
from spectacle.modeling.fitting import LevMarFitter, LevMarLSQFitter, DynamicLevMarFitter

# Misty
import os, sys
# sys.path.insert(0, "/Users/nearl/projects/enzo_specs/")
# import MISTY

# yt/enzo
import trident
import yt

# general
import numpy as np
import astropy as at
import matplotlib.pyplot as plt

%matplotlib notebook
print(np.__version__, at.__version__)

INFO:root:Added misty to custom loaders.


1.12.1 1.3.2


# Trident setup

In [6]:
# Load trident ray data using yt
tri_ray = yt.load("/Users/nmearl/Downloads/RD0042_ray_z_imp177.9_ang1.30_tri.h5")
# tri_ray = yt.load("/Users/nearl/Downloads/RD0042_ray_z_imp020.0_ang0.00tri.h5")
# tri_ray = yt.load("/Users/nearl/Downloads/RD0042_ray_z_imp046.4_ang3.23_tri.h5")
# tri_ray = yt.load("/Users/nearl/Downloads/RD0042_ray_z_imp063.8_ang6.07_tri.h5")
# tri_ray = yt.load("/Users/nearl/Downloads/RD0042_ray_z_imp022.9_ang5.32_tri.h5")

# Parse the start/end of ray
ray_start = tri_ray.light_ray_solution[0]['start']
ray_end = tri_ray.light_ray_solution[0]['end']

# Get line information for the line we want
line_name = "Si II 1260"
ldb = trident.LineDatabase('lines.txt')
line_out = ldb.parse_subset(line_name)
line_out = line_out[0]
ar = tri_ray.all_data()

print(line_name, line_out)

# Parse spectrum information from ray
lambda_rest = line_out.wavelength
lambda_min = lambda_rest * (1+min(ar['redshift_eff'])) - 0
# lambda_max = lambda_rest * (1+max(ar['redshift_eff'])) + 0
# sg = trident.SpectrumGenerator(lambda_min=lambda_min.value, lambda_max=lambda_max.value, dlambda=0.0001)
# sg.make_spectrum(tri_ray, lines=line_out.name)

yt : [INFO     ] 2017-06-29 17:22:50,713 Parameters: current_time              = 4.3430653996e+17
yt : [INFO     ] 2017-06-29 17:22:50,713 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-29 17:22:50,715 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-29 17:22:50,716 Parameters: domain_right_edge         = [  4.43982386e+26   4.43982386e+26   4.43982386e+26]
yt : [INFO     ] 2017-06-29 17:22:50,718 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-29 17:22:50,719 Parameters: current_redshift          = 4.4408920985e-16
yt : [INFO     ] 2017-06-29 17:22:50,720 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2017-06-29 17:22:50,722 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2017-06-29 17:22:50,723 Parameters: hubble_constant           = 0.695
yt : [INFO     ] 2017-06-29 17:22:50,849 Allocating for 5.800e+01 particles (index particle type 'all')
yt : [INFO     ] 2017-06-29 17:22:50,8

Si II 1260 Si II 1260


FieldUnitsError: Cannot handle units 'b'cm**(-3)'' (type <class 'bytes'>).Please provide a string or Unit object.

# Spectacle setup

In [ ]:
# Create Spectrum1D from trident information
disp = np.array(list(sg.lambda_field))
flux = np.array(list(sg.flux_field))
spectrum = Spectrum1D(flux, dispersion=disp)

In [ ]:
# Plot the trident spectrum
f, ax1 = plt.subplots()

ax1.plot(disp, flux)

In [ ]:
# Create Line1D from trident line information
sg_line = sg.line_list[0]

# This process will find lines in the trident spectrum
# and assign the values set in the `defaults` dict to
# the new lines found.
lines = spectrum.find_lines(threshold=0.02/max(1-spectrum.data),
                            min_dist=10,
                            smooth=True,
                            interpolate=True,
                            defaults=dict(
                                lambda_0=sg_line['wavelength'].value,
                                f_value=sg_line['f_value'],
                                gamma=sg_line['gamma'],
                                fixed={
                                    'lambda_0': True,
                                    'delta_v': True,
                                    'delta_lambda': False}
                           ))

# Create absorption Spectrum1D from line information
spec_mod = Absorption1D(lines=lines)

# Create a Spectrum1D object from the Absorption1D model
gen_spec = spec_mod(spectrum.dispersion)

In [ ]:
# Plot the generated absorption spectrum
f, ax = plt.subplots()

ax.plot(spectrum.dispersion, spectrum.data, label="Original")
ax.plot(spectrum.dispersion, gen_spec.data, label="Generated")
# ax.set_xlim(1259.63, 1260.78)

plt.legend(loc=0)

In [ ]:
# Now fit the generated Spectrum1D object to the ray spectrum data

print(spectrum.dispersion.dtype, spectrum.data.dtype)

# Create a fitter
fitter = DynamicLevMarFitter()

fit_spec_mod = fitter(spec_mod, spectrum.dispersion, spectrum.data,
                      maxiter=1000, initialize=False)

# Get the results of the fit
fit_spec = fit_spec_mod(spectrum.dispersion)

In [ ]:
print(type(fit_spec_mod))

# Plot the stacked results
f, (ax1) = plt.subplots()

ax1.step(spectrum.dispersion, spectrum.data, label="Data")
# ax1.step(gen_spec.dispersion, gen_spec.data, label="Initial model")
ax1.step(fit_spec.dispersion, fit_spec.data, label="Fitting result")

for sub_mod in fit_spec_mod[1:]:
    fit_sub_spec = (sub_mod + fit_spec_mod[0])(spectrum.dispersion)
    
    ax1.plot(spectrum.dispersion, fit_sub_spec, linestyle='--', alpha=0.5)

# ax1.set_xlim(1259.7, 1260.26)

plt.legend(loc=0)